# Übung 7: Regularisierung und  Decision Trees

#### Aufgabe 1

In der Vorlesung haben wir gelernt, dass die L1 Regulierung dazu neigt viele Koeffizieten auf 0 zu setzen. 

Benutzen Sie die Daten der männlichen Raucher und führen Sie darauf eine Regression mit unterschiedlich starker Regularisierung, sowohl L1 als auch L2, durch. Schauen Sie sich dabei verschiedene statistische Kennzahlen der Regressionskoeffizieten wie Mittelwert, Min/Max und Quantile an. Zählen Sie wie viele Koeffizieten dabei 0 sind. Was stellen Sie fest?

#### Aufgabe 2

Nehme Sie nun alle Daten aus der 'insurance.csv' und trainieren Sie eine Regression mit einem DecisionTree. Variieren Sie der Parameter 'max_depth' und 'min_sample_leaves' und suchen Sie die beste Kombination.

#### Aufgabe 3

Wie Sie in Aufgabe 2 gesehen haben ist es sehr aufwändig die richtigen Parameter zu finden. Zum Glück kann scikit-learn dies automatisieren. Lesen Sie sich dafür in 'GridSearchCV' ein und finden Sie die besten Parameter für 'max_depth', 'min_samples_split' und 'min_samples_leaf'.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, PolynomialFeatures

In [ ]:
insurance = pd.read_csv("data/insurance.csv")
insurance.head()

In [ ]:
male_smokers = insurance[
    (insurance["sex"] == "male") & (insurance["smoker"] == "yes")
].sort_values(by=["bmi"])

X = male_smokers["bmi"].to_numpy().reshape(-1, 1)
y = male_smokers["expenses"].to_numpy().reshape(-1, 1)

## Aufgabe 1

In [ ]:
# benutzen Sie np.abs(coeff) < eps statt wirklich nach 0 zu suchen
eps = 1e-8

poly_features = PolynomialFeatures(degree=50, include_bias=False)
std_scaler = MinMaxScaler()
# lam muss in der Schleife gesetzt werden
lam = None
ridge_reg = Ridge(alpha=lam)
pipe_ridge = Pipeline(
    [
        ("std_scaler", std_scaler),
        ("poly_features", poly_features),
        ("ridge", ridge_reg),
    ]
)
# Parameter der ridge regression aus der pipeline
# pipe_lasso.get_params()["ridge"].coef_

# analog zu oben
# pipe_ridge = ...

# Schleife über lam(bda)

## Aufgabe 2

In [ ]:
X = insurance.drop(columns=["expenses"], axis=1)
y = insurance["expenses"].to_numpy().reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

numeric_features = ["age", "bmi", "children"]
ordinal_features = ["sex", "smoker"]

numeric_transformer = Pipeline(  # Decision Tree braucht kein Scaling
    steps=[
        (
            "imputer",
            SimpleImputer(strategy="mean"),
        ),
        ("poly_features", PolynomialFeatures(degree=8)),
    ]
)

ordinal_transfomer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ordinal_encoding", OrdinalEncoder()),
    ]
)

y_preprocessor = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
    ]
)

X_preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat_ordinal", ordinal_transfomer, ordinal_features),
    ]
)


X_train_prepared = X_preprocessor.fit_transform(X_train)
y_train_prepared = y_preprocessor.fit_transform(y_train)

# Doppelschleife über max_depth und min_sample leaves

## Aufgabe 3

In [ ]:
from sklearn.model_selection import GridSearchCV

square = lambda n: [x**2 for x in np.arange(1, n)]
param_grid = {
    "max_depth": square(10),
    "min_samples_split": square(10),
    "min_samples_leaf": square(10),
}

In [ ]:
clf = DecisionTreeRegressor(**best_params)
clf.fit(X_train_prepared, y_train_prepared)

print(f"Train score: {clf.score(X_train_prepared, y_train_prepared)}")
print(f"Test score: {clf.score(X_test_prepared, y_test_prepared)}")